# 📊 Bank Statement Analyzer

Parse • Clean • Analyze Indian Bank Statement PDFs

## Overview
This notebook analyzes Indian bank statement PDFs and generates clean CSV outputs.

## Configuration

In [ ]:

BANK= "HDFC"  # Options: HDFC | KOTAK
PDF_PATH = "sample_statement.pdf"
OUTPUT_CSV = "merchant_summary.csv"


## Imports

In [ ]:

import re
import os
import pandas as pd
import numpy as np
import pdfplumber
import matplotlib.pyplot as plt
from typing import Dict
import pandas as pd
from openpyxl import load_workbook


## Regex For Accounts

In [ ]:
KOTAK_TXN_REGEX = re.compile(
    r"(?P<date>\d{2}-\d{2}-\d{4})\s+"
    r"(?P<desc>.*?)\s+"
    r"(?P<ref>(?:UPI|NEFTINW|NEFT)-[\w\d]+)\s+"
    r"(?P<amount>[\d,]+\.\d{2})\((?P<drcr>Dr|Cr)\)\s+"
    r"(?P<balance>[\d,]+\.\d{2})\((?:Dr|Cr)\)",
    re.IGNORECASE
    )



HDFC_TXN_REGEX = re.compile(
    r"(?P<date>\d{2}/\d{2}/\d{2})\s+"
    r"(?P<desc>.*?)\s+"
    r"(?P<ref>[A-Z0-9]+)\s+"
    r"(?P<value_dt>\d{2}/\d{2}/\d{2})\s+"
    r"(?P<amount>[\d,]+\.\d{2})\s+"
    r"(?P<balance>[\d,]+\.\d{2})",
    re.IGNORECASE
)

## Processing Pipeline
PDF → Transactions → Merchant Cleaning → Aggregation → CSV

## 1. Bank Extraction

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


## Kotak Extractor

In [ ]:
def extract_kotak_transactions_table(text):
    records = []

    for m in KOTAK_TXN_REGEX.finditer(text):
        withdrawal = m.group("withdrawal")
        deposit = m.group("deposit")

        debit = float(withdrawal.replace(",", "")) if withdrawal else 0
        credit = float(deposit.replace(",", "")) if deposit else 0

        records.append({
            "date": pd.to_datetime(m.group("date"), dayfirst=True),
            "description": m.group("desc").strip(),
            "reference": m.group("ref"),
            "debit": debit,
            "credit": credit,
            "transaction_amount": credit - debit,
            "balance": float(m.group("balance").replace(",", "")),
            "txn_type": "CREDIT" if credit > 0 else "DEBIT"
        })

    return pd.DataFrame(records)


In [ ]:
def extract_kotak_transactions_inline(text: str) -> pd.DataFrame:
    """
    Extract transactions from Kotak statements where
    amounts and balances are marked inline with (Dr)/(Cr)
    and narration spans multiple lines.
    """

    records = []

    # Step 1: split into logical transaction blocks
    blocks = []
    current_block = []

    for line in text.splitlines():
        line = line.strip()

        # Transaction starts with date like 01-10-2025
        if re.match(r"\d{2}-\d{2}-\d{4}", line):
            if current_block:
                blocks.append(" ".join(current_block))
                current_block = []

        if line:
            current_block.append(line)

    if current_block:
        blocks.append(" ".join(current_block))

    # Step 3: parse each block
    for block in blocks:
        match = KOTAK_TXN_REGEX.search(block)
        if not match:
            continue

        amount = float(match.group("amount").replace(",", ""))
        balance = float(match.group("balance").replace(",", ""))
        drcr = match.group("drcr").upper()

        debit = amount if drcr == "DR" else 0.0
        credit = amount if drcr == "CR" else 0.0

        records.append({
            "date": pd.to_datetime(match.group("date"), dayfirst=True),
            "description": match.group("desc").strip(),
            "reference": match.group("ref"),
            "debit": debit,
            "credit": credit,
            "transaction_amount": credit - debit,
            "balance": balance,
            "txn_type": "CREDIT" if credit > 0 else "DEBIT"
        })

    return pd.DataFrame(records)



In [ ]:
def detect_kotak_format(text: str) -> str:
    if "(Dr)" in text and "(Cr)" in text:
        return "INLINE_DRCR"
    if "Withdrawal" in text and "Deposit" in text:
        return "TABLE"
    raise ValueError("Unknown Kotak format")


In [ ]:
def extract_kotak_transactions(text: str) -> pd.DataFrame:
    fmt = detect_kotak_format(text)

    if fmt == "TABLE":
        return extract_kotak_transactions_table(text)

    if fmt == "INLINE_DRCR":
        return extract_kotak_transactions_inline(text)


## HDFC Extractor

In [ ]:
def split_hdfc_transaction_blocks(text: str):
    blocks = []
    current = []

    for line in text.splitlines():
        line = line.strip()

        # HDFC transaction rows start with date dd/mm/yy
        if re.match(r"\d{2}/\d{2}/\d{2}", line):
            if current:
                blocks.append(" ".join(current))
                current = []

        if line:
            current.append(line)

    if current:
        blocks.append(" ".join(current))

    return blocks


In [ ]:
def extract_hdfc_transactions(text: str) -> pd.DataFrame:
    records = []
    prev_balance = None

    blocks = split_hdfc_transaction_blocks(text)

    for block in blocks:
        m = HDFC_TXN_REGEX.search(block)
        if not m:
            continue

        amount = float(m.group("amount").replace(",", ""))
        balance = float(m.group("balance").replace(",", ""))

        if prev_balance is None:
            # FIRST ROW — infer from narration keywords
            if "CR" in block.upper():
                credit, debit = amount, 0.0
            else:
                debit, credit = amount, 0.0
        else:
            if balance > prev_balance:
                credit, debit = amount, 0.0
            else:
                debit, credit = amount, 0.0

        records.append({
            "date": pd.to_datetime(m.group("date"), dayfirst=True),
            "description": m.group("desc").strip(),
            "reference": m.group("ref"),
            "debit": debit,
            "credit": credit,
            "transaction_amount": credit - debit,
            "balance": balance,
            "txn_type": "CREDIT" if credit > 0 else "DEBIT",
        })

        prev_balance = balance

    return pd.DataFrame(records)


## 2. Merchant Extraction & Normalisation

In [ ]:
def extract_merchant(description: str) -> str:
    if not isinstance(description, str):
        return "UNKNOWN"

    d = description.strip().upper()

    # -------------------------------------------------
    # UPI (Kotak + HDFC)
    # UPI/merchant/...  OR  UPI-merchant-...
    # -------------------------------------------------
    if d.startswith("UPI"):
        cleaned = d.replace("/", "-")
        parts = cleaned.split("-")

        # UPI-merchant-...
        if len(parts) >= 2 and parts[1]:
            return parts[1].strip()

        return "UPI"

    # -------------------------------------------------
    # NEFT (Kotak style)
    # NEFT <IFSC/UTR> <BENEFICIARY NAME> <BANK CODE>
    # -------------------------------------------------
    if d.startswith("NEFT"):
        # Normalize separators
        cleaned = d.replace("/", "-")
        parts = [p.strip() for p in cleaned.split("-") if p.strip()]

        # Example parts:
        # ['NEFT CR', 'TXNID', 'MERCHANT NAME', 'HDFC0000001']
        # ['NEFT DR', 'TXNID', 'MERCHANT NAME', 'SBI0000123']

        if len(parts) >= 3:
            return parts[2]

        return "NEFT_TRANSFER"
    # -------------------------------------------------
    # IMPS (similar logic to NEFT)
    # -------------------------------------------------
    if d.startswith("IMPS"):
        parts = d.split()
        if len(parts) >= 4:
            beneficiary = " ".join(parts[2:-1])
            return beneficiary.strip()
        return "IMPS_TRANSFER"

    # -------------------------------------------------
    # POS
    # -------------------------------------------------
    if d.startswith("POS"):
        parts = d.split()
        return parts[1] if len(parts) > 1 else "POS"

    # -------------------------------------------------
    # Fallback
    # -------------------------------------------------
    return d.split()[0]


In [ ]:

MERCHANT_ALIASES = {
    "ZOMATO/ZEPTO/SWIGGY": [
        "ZOMATO",
        "BLINKIT",
        "BISTRO",
        "GROFERSINDIAPRIVAT",
        "ZEPTOMARKETPLACEPR",
    ],
    "PAYTM": [
        "PAYTM",
        "ONE97",
    ],
    "Subscriptions": [
        "APPLEMEDIASERVICES",
    ],
    "Family": [
    ],

    "Friends": [
    ],

    "Markup&deduction": [
        
    ],

    "Selftransafer":[
       
    ],
}



In [ ]:
def normalize_merchant(merchant: str) -> str:
    if not isinstance(merchant, str):
        return "UNKNOWN"

    m = merchant.upper().strip()

    for canonical, aliases in MERCHANT_ALIASES.items():
        for alias in aliases:
            # Prefix match (for terminal / branch codes)
            if m.startswith(alias):
                return canonical

            # Substring match
            if alias in m:
                return canonical

    # 🔹 No alias matched → return original merchant (cleaned)
    return m


## 3. Enforce credit consistency

In [ ]:
def enforce_debit_credit_consistency(df: pd.DataFrame) -> pd.DataFrame:
    """
    Enforces:
    - Debit  = money OUT (positive debit)
    - Credit = money IN  (positive credit)
    - transaction_amount = credit - debit
    - txn_type derived ONLY from amounts
    """

    df = df.copy()

    # Ensure no negatives inside debit / credit columns
    df["debit"] = df["debit"].fillna(0).clip(lower=0)
    df["credit"] = df["credit"].fillna(0).clip(lower=0)

    # Recalculate transaction amount
    df["transaction_amount"] = df["credit"] - df["debit"]

    # Re-derive txn_type (single source of truth)
    df["txn_type"] = df["transaction_amount"].apply(
        lambda x: "CREDIT" if x > 0 else "DEBIT"
    )

    return df

## 4. Pipeline Selection

In [ ]:
BANK_PIPELINES: Dict[str, callable] = {
    "kotak": extract_kotak_transactions,
    "hdfc": extract_hdfc_transactions
}

In [ ]:
def extract_transactions_pipeline(pdf_path: str, bank: str) -> pd.DataFrame:
    text = extract_text_from_pdf(pdf_path)

    if bank not in BANK_PIPELINES:
        raise ValueError("Unsupported bank")

    df = BANK_PIPELINES[bank](text)

    if df.empty:
        raise ValueError("No transactions extracted — check regex")

    df["merchant_raw"] = df["description"].apply(extract_merchant)
    df["merchant"] = df["merchant_raw"].apply(normalize_merchant)

    return df


## Output
Generated CSV files are ready for Excel or BI tools.

In [ ]:
df = extract_transactions_pipeline(PDF_PATH,BANK)
df = enforce_debit_credit_consistency(df)
df.head(50)

In [ ]:
def create_merchant_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates a merchant-level summary from transaction dataframe
    """

    summary = (
        df.groupby("merchant", dropna=False)
          .agg(
              total_debit=("debit", "sum"),
              total_credit=("credit", "sum"),
              net_amount=("transaction_amount", "sum"),
              transaction_count=("transaction_amount", "count"),
              first_transaction=("date", "min"),
              last_transaction=("date", "max"),
          )
          .reset_index()
          .sort_values(by="total_debit", ascending=False)
    )

    return summary


In [ ]:
merchant_summary = create_merchant_summary(df)
merchant_summary.head(10)


In [ ]:
merchant_summary.to_csv(OUTPUT_CSV, index=False)
print("✅ merchant_summary.csv saved")

## Conclusion
Extend this notebook by adding new bank parsers or analysis layers.